In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
!pip install flask
!pip install pyngrok
!pip install transformers
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 68.7 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.2 MB/s eta 0:00:00:00:0100:01


In [3]:
from flask import Flask, request, jsonify
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from pyngrok import ngrok
import re

# Initialize Flask application
app = Flask(__name__)

# Set the model name
model_name = "Ayushx29/finance_finetune_model"

# Cache the model and tokenizer loading
def load_model_and_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return model, tokenizer

# Load the model and tokenizer
model, tokenizer = load_model_and_tokenizer(model_name)

# Function to generate text based on input prompt
def generate_text(prompt, max_length=100):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length)
    result = pipe(f"<s>[INST] {prompt} [/INST]")[0]['generated_text']
    # Remove <s>[INST] ... [/INST] from the result
    result = re.sub(r"<s>\[INST\].*?\[/INST\]", "", result).strip()
    return result

# Route for generating text based on input prompt
@app.route('/generate', methods=['POST'])
def generate():
    data = request.json
    prompt = data.get('prompt')
    response_text = generate_text(prompt)
    return jsonify({'generated_text': response_text})

if __name__ == '__main__':
    try:
        # Start ngrok and get the public URL
        ngrok.set_auth_token('2vFxFXwbkbcH5lpb5uxGcTVJzA3_42FngpBtqYhucbsinTGE3')
        http_tunnel = ngrok.connect(addr='5000', proto='http')
        print(f"Public URL: {http_tunnel.public_url}")
        
        # Run the Flask app
        app.run(host='0.0.0.0', port=5000)
    
    except Exception as e:
        print(f"Error starting ngrok or Flask application: {str(e)}")

2025-06-21 10:27:09.980784: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750501630.230413      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750501630.301600      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Public URL: https://e45d-104-198-12-128.ngrok-free.app                                              
 * Serving Flask app '__main__'
 * Debug mode: off


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Device set to use cpu
